# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("./data/song_data")

In [6]:
filepath = song_files[0]

In [7]:

df = pd.DataFrame(pd.read_json(filepath, lines=True, orient='columns'))
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
# song ID, title, artist ID, year, and duration
song_data = (df.values[0][7], df.values[0][8], df.values[0][0], df.values[0][9], df.values[0][5])
song_data

('SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
#NOTE: To prevent duplicates use "ON CONFLICT (song_id) DO NOTHING" in SQL query.
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
# artist ID, name, location, latitude, and longitude
artist_data = (df.values[0][0], df.values[0][4], df.values[0][2], df.values[0][1], df.values[0][3])
artist_data

('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
# #NOTE: To prevent duplicates use "ON CONFLICT (artist_id) DO NOTHING" in SQL query.
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("./data/log_data")

In [13]:
filepath = log_files[0]

In [14]:
df = pd.DataFrame(pd.read_json(filepath, lines=True, orient='columns'))
df_orig = df
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75
6,Gary Allan,Logged In,Ryann,F,0,Smith,259.83955,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,The One,200,1541930188796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
7,Miracle Fortress,Logged In,Ryann,F,1,Smith,200.90730,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Five Roses,200,1541930447796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
8,Don Omar,Logged In,Ryann,F,2,Smith,261.35465,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Cuentale,200,1541930647796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
9,Jay-Z,Logged In,Ryann,F,3,Smith,212.27057,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,D'Evils,200,1541930908796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [31]:
# See: https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/
# "filter rows for year 2002 using  the boolean variable"
# df['page']=='NextSong' tells if record has NextSong value in page columns (true/false).
# df[df['page']=='NextSong'] filters those records == leaves out records having some other value.
df = df[df['page']=='NextSong']
#df.head()

In [27]:
t = pd.to_datetime(df['ts'], unit='ms')
#t.head()


In [20]:
# See: https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
# See: https://stackoverflow.com/questions/44741587/pandas-timestamp-series-to-string
time_data = list(zip(t.dt.strftime('%Y-%m-%d %I:%M:%S'), t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
#time_data

In [29]:
time_df = pd.DataFrame(time_data, columns=column_labels)

for i, row in time_df.iterrows():
    print(list(row))

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [30]:
user_data = df_orig.get(['userId', 'firstName', 'lastName', 'gender', 'level'])
# adjust column names
user_data.columns = ['user_id', 'first_name', 'last_name', 'gender', 'level']
# remove rows with no user_id
user_data_clean = user_data[user_data['user_id']!= '']
# remove duplicates
user_data_duplicates_removed = user_data_clean.drop_duplicates('user_id', keep='first')

user_df = user_data_duplicates_removed
#user_df

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
# #NOTE: To prevent duplicates use "ON CONFLICT (user_id) DO NOTHING" in SQL query.
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
# NOTE: In Postgresql, use %s as a string operator for incoming variables
# e.g. INSERT ...VALUES (%s, %s) or SELECT ...WHERE s.name = %s AND a.artist_id = %s AND s.length = %s
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    #print(songid, artistid)
    
    start_time = pd.to_datetime(row.ts, unit='ms').strftime('%Y-%m-%d %I:%M:%S')
    songplay_data = (start_time, row.userId, row.level, str(songid), str(artistid), row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.